# Sonar Simulator Demo

In [ ]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
import time

from demo_funcs import *

## Reduced Model Visualization

In [ ]:
demo = load_demo_data()
plot_stability_and_energy(demo)

Options: 'cont', 'pulse', '5hz'

In [ ]:
signals = ['cont', 'pulse']

plot_all_inputs(signals)

Visualize

In [ ]:
#%matplotlib qt
animate_2_waveforms_error(demo, signals)

In [ ]:
# Save as GIF
#animate_2_waveforms_error(demo, signals, n_frames=100, fps=15, save_gif='demo_outputs/rom_animation_5hz.gif')

In [ ]:
plot_snapshots(demo, 'cont', n_snapshots=3)  # Default 6 snapshots

In [ ]:
plot_error_over_time(demo, 'cont')

In [ ]:
demo = load_demo_data()

Phi = demo['Phi']
N = demo['N']

print("Total L2 Relative Error by Waveform")
print("=" * 40)

for waveform in ['cont', 'pulse', '5hz']:
    if waveform not in demo['full_order']:
        continue
    if demo['full_order'][waveform]['X'] is None:
        continue
    
    # Simulate ROM
    rom_result = simulate_rom(demo, waveform)
    
    # Get data
    X_full = demo['full_order'][waveform]['X']
    x_rom = rom_result['x']
    
    # Reconstruct and extract pressure
    X_rom = (Phi @ x_rom.T)
    p_full = X_full[N:, :]
    p_rom = X_rom[N:, :]
    
    # Truncate to same length
    n_steps = min(p_full.shape[1], p_rom.shape[1])
    p_full = p_full[:, :n_steps]
    p_rom = p_rom[:, :n_steps]
    
    # Frobenius norm error
    total_error = np.linalg.norm(p_full - p_rom, 'fro')
    total_norm = np.linalg.norm(p_full, 'fro')
    rel_err = total_error / total_norm * 100
    
    print(f"  {waveform}: {rel_err:.4f}%")

print("=" * 40)